<a href="https://colab.research.google.com/github/AmatureJS/NPS-Retail-BanK/blob/main/Net_Promotor_Score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install plotly

In [18]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [14]:
colourlist = ['#871e71','#b42479','#da3877','#eb676e','#f79473','#fbd395']

In [ ]:
# reading file
data = pd.read_csv("/content/NPStimeseries.csv")

# it will should details of customer
data.head()

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             5000 non-null   int64 
 1   Market         5000 non-null   object
 2   Survey date    5000 non-null   object
 3   Customer Name  5000 non-null   object
 4   Month          5000 non-null   int64 
 5   Quarter        5000 non-null   int64 
 6   NPS            5000 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 273.6+ KB
None


In [8]:
conditions = [
    (data['NPS'] <= 6),
    ((data['NPS'] >6 )& (data['NPS'] < 9)),
    (data['NPS'] >= 9)
]

values = ['Detractor', 'Passive', 'Promoter']

data['NPS category'] = np.select(conditions, values)
print(data.head())


     ID Market Survey date    Customer Name  Month  Quarter  NPS NPS category
0  1000     US  01/09/2021  Krista Richards      9        3   10     Promoter
1  1001    MEX  07/11/2021      Monica King     11        4    9     Promoter
2  1002     UK  25/12/2021  Ricky Armstrong     12        4    0    Detractor
3  1003     UK  01/10/2021     Andrea Foley     10        4   10     Promoter
4  1004     UK  01/03/2021     Jerry Garcia      3        1    8      Passive


In [20]:
# Assuming 'data' is your original DataFrame with relevant columns
# Grouping the data by 'Market' and 'NPS category' and counting 'ID'
dfmarket = data.groupby(['Market', 'NPS category'])['ID'].count()

# Unstacking to create a DataFrame with Markets as rows and NPS categories as columns
dfmarket = dfmarket.unstack(fill_value=0)  # Use fill_value to handle missing values

# Adding a 'Total' column that sums across NPS categories
dfmarket['Total'] = dfmarket.sum(axis=1)

# Resetting index to have 'Market' as a column
dfmarket = dfmarket.reset_index()

# Calculating the NPS score
# Ensure the categories match the actual column names after unstacking
dfmarket['NPS score'] = (
    dfmarket.get('Promoter', 0) / dfmarket['Total'] -
    dfmarket.get('Detractor', 0) / dfmarket['Total']
) * 100

# Dropping unnecessary columns
dfmarket = dfmarket.drop(columns=['Detractor', 'Passive', 'Promoter', 'Total'], errors='ignore')

#print(dfmarket)

# Plotting the NPS score by market using Plotly Express
fig = px.bar(
    dfmarket,
    x='Market',
    y='NPS score',
    title='NPS Score by Market',
    template='plotly_white',
    color='Market',
    color_discrete_map={'MEX': '#871e71', 'UK': '#da3877', 'US': '#fbd395'}
)

# Displaying the figure
fig.show()